In [1]:
import numpy  as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

### Data

In [2]:
df = pd.read_csv("../data/macros.csv", index_col="Uniprot Code")
df["Longitud"] = df.Secuencia.str.len() # Add lenght column
df.head()

,Tipo de Macro,Secuencia,Longitud
Uniprot Code,,,
O28751,AF-1521-like,MEVLFEAKVGDITLKLAQGDITQYPAKAIVNAANKRLEHGGGVAYA...,192
D3RWS7,AF-1521-like,MEVEVVRELEMDKLKVKLAGGDITKYPAEAIVNAANKYLEHGGGVA...,193
D2RH24,AF-1521-like,MVVKKFGSVEVVLEKGDITKYPAEAIVNAANKYLEHGGGVALAIAK...,193
A0A0F7ICE9,AF-1521-like,MKPEVVLRFSGVEVRLVQGDITKYPAEAIVNAANRHLEHGGGVAYA...,194
A0A075LQ95,AF-1521-like,MNLTELTFGNLTFKLAQGDITKLPAEAIVNAANKYLEHGGGVALAI...,190


In [3]:
total_num_aminoacids = df["Longitud"].sum()
total_num_aminoacids

182079

### Tokenizer

In [4]:
aminoacids = sorted(list(set(df['Secuencia'].apply(set).apply(list).sum())))
aminoacids = aminoacids + ["[PAD]"]
vocab_size = len(aminoacids)

amin_dict = {a: i for i, a in enumerate(aminoacids)}
numb_dict = {i: a for i, a in enumerate(aminoacids)}
amin_dict

{'A': 0,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'K': 8,
 'L': 9,
 'M': 10,
 'N': 11,
 'P': 12,
 'Q': 13,
 'R': 14,
 'S': 15,
 'T': 16,
 'V': 17,
 'W': 18,
 'X': 19,
 'Y': 20,
 '[PAD]': 21}

### DataLoader
Predecir acada aminoacido por sus antariores y posteriores

In [5]:
class SequencesDataset(torch.utils.data.Dataset):
    
    def __init__(self, n_anteriores=4, n_posteriores=4):
        indexes = []
        
        # Paddin at the beggining
        for i in range(n_anteriores):
            indexes.append([-1,amin_dict["[PAD]"]])
        
        # Save [seq_id, aminoacid_number] pairs FOR ALL AMINOACIDS
        for i, seq in enumerate(df.Secuencia):
            for ami in seq:
                indexes.append([i, amin_dict[ami]])

        # Padding at the end
        for i in range(n_posteriores):
            indexes.append([-1,amin_dict["[PAD]"]])
    

        self.indexes       = np.array(indexes)
        self.n_anteriores  = n_anteriores
        self.n_posteriores = n_posteriores

    def __getitem__(self, index):
        
        # Add anteriores to select correct index
        index += self.n_anteriores
        
        # Select window
        window = self.indexes[index-self.n_anteriores:index+self.n_posteriores+1]
        
        # Find sequence id, and target aminoacid
        seq_num = window[self.n_anteriores, 0]
        ami_y   = window[self.n_anteriores, 1]
        
        # Delete target aminoacid from window
        window = np.delete(window, self.n_anteriores, axis=0)
        
        # Padding if some window aminoacid is from another sequence
        window[:,1][window[:,0]!=seq_num] = amin_dict["[PAD]"]
        
        # Select aminacids from window (anteriores y posteriores)
        ami_x  = window[:, 1]
        
        # To tensor
        ami_x = torch.tensor(ami_x)
        ami_y = torch.tensor(ami_y)
        
        
        return ami_x, ami_y
    
    def __len__(self):
        return total_num_aminoacids

### Parameters

In [6]:
n_anteriores  = 4
n_posteriores = 4
n_inputs      = n_anteriores + n_posteriores
emb_size      = 8 # (less than 20, because ther are 20 types of aminoacids)
n_hidden      = 100
n_output      = vocab_size

dataset    = SequencesDataset(n_anteriores, n_posteriores)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
dataset[0]

(tensor([21, 21, 21, 21,  3, 17,  9,  4]), tensor(10))

### Model

In [7]:
dtype = torch.FloatTensor

class LM(nn.Module):
    def __init__(self):
        super(LM, self).__init__()
        self.E  = nn.Embedding(vocab_size, emb_size)                                   # Embedding
        self.W1 = nn.Parameter(torch.randn(n_inputs * emb_size, n_hidden).type(dtype)) # Dense 1 weights
        self.B1 = nn.Parameter(torch.randn(n_hidden).type(dtype))                      # Dense 1 bias
        self.W2 = nn.Parameter(torch.randn(n_hidden, n_output).type(dtype))            # Dense 2 weights
        self.RW = nn.Parameter(torch.randn(n_inputs * emb_size, n_output).type(dtype)) # Dense 2 residual weights
        self.B2 = nn.Parameter(torch.randn(n_output).type(dtype))                      # Dense 2 bias

    def forward(self, X):
        X = self.E(X)                       # Embeding layer          [bs, n_inputs,  emb_size]
        X = X.view(-1, n_inputs * emb_size) # Embedings concatenation [bs, n_inputs * emb_size]
        tanh = torch.tanh(self.B1 + torch.mm(X, self.W1)) # Dense layer 1 [bs, hidden_size]
        output = self.B2 + torch.mm(X, self.RW) + torch.mm(tanh, self.W2) # Dense layer 2 with residual [bs, vocab_size]
        return output

model = LM()

### Train

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1):

    for i, batch in enumerate(dataloader):
        
        inputs, labels = batch      # Get [x,y] data
        optimizer.zero_grad()       # Zero the parameter gradients
        output = model(inputs)      # Forward -> [batch_size, vocab_size]
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1)%1000 == 0:
            print('Step:', '%04d' % (i + 1), 'cost =', '{:.6f}'.format(loss))

Step: 1000 cost = 15.247977
Step: 2000 cost = 8.468339
Step: 3000 cost = 6.261123
Step: 4000 cost = 6.773494
Step: 5000 cost = 5.099769
Step: 6000 cost = 4.200829
Step: 7000 cost = 3.520826
Step: 8000 cost = 4.982923
Step: 9000 cost = 4.239739
Step: 10000 cost = 3.083456
Step: 11000 cost = 3.351819


### Plot embeddings

In [9]:
emb_df = pd.DataFrame(data=model.E.weight.data.numpy(), index=aminoacids)
emb_df = emb_df.drop(["X", "[PAD]"])

from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit_transform(emb_df.to_numpy())  # Compute PCA
emb_df['pca1'] = pca[:,0]
emb_df['pca2'] = pca[:,1]
emb_df

,0,1,2,3,4,5,6,7,pca1,pca2
A,0.283717,0.195669,0.473640,0.106540,0.716131,-0.021891,0.142731,0.104561,-0.196755,-0.118723
C,0.160171,0.390802,0.438237,-0.204678,0.592143,0.140152,-0.091551,0.421034,-0.530339,0.035108
D,0.423275,-0.106441,0.344730,0.453353,0.327267,-0.136446,0.356835,0.403207,0.387629,-0.448104
E,0.146869,0.590459,0.459565,0.060844,0.891540,0.057335,0.098626,-0.093883,-0.387901,0.296791
F,0.208693,0.217312,0.509949,-0.068511,0.664986,0.044335,0.031112,0.277070,-0.373452,-0.107069
G,0.154652,0.713851,0.393957,-0.015624,0.828595,0.218472,-0.045019,-0.043033,-0.512101,0.432557
H,-0.326036,1.013176,0.185688,0.814158,0.327890,0.568581,0.697741,-0.252586,0.645722,1.238576
I,0.172370,0.309893,0.488456,0.048835,0.648789,0.067857,0.181176,0.156057,-0.213514,0.040688
K,0.099727,0.682186,0.483253,0.025245,0.871173,0.168764,-0.010429,-0.086051,-0.487728,0.419298
L,0.560810,-0.228176,0.448011,0.520682,0.808180,-0.345476,0.475233,-0.133881,0.326914,-0.574263


In [10]:
import altair as alt

base = alt.Chart(emb_df[['pca1','pca2']].reset_index()).encode(
    x='pca1', y='pca2'
)

base.mark_circle() + base.mark_text(dx=10).encode(text='index')

alt.LayerChart(...)

### Predict

In [ ]:
# Predict
predict = model(input_batch).data.max(1, keepdim=True)[1]

# Test
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])